### It happens all the time: someone gives you data containing malformed strings,
### Python, lists and missing data. How do you tidy it up so you can get on with the analysis?


### Take this monstrosity as the DataFrame to use in the following puzzles:

In [136]:
import pandas as pd
import numpy as np
import re

In [137]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

### 1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [138]:
df['FlightNumber']=[i for i in range(10045,10086,10)]

### 2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

### 3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [139]:
def from_to_cleaning(column_name):
    
    From=list()
    To=[]
    try:
        for item in column_name:
            i1,i2=item.split('_')
            From.append(i1.lower().title())
            To.append(i2.lower().title())

        return (From,To)
    except Exception as e:
        print(e)
From,To=from_to_cleaning(df['From_To'])
new_df=pd.DataFrame({'From':From,'To':To})


### 4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [140]:
df.drop(columns='From_To',inplace=True)
df=df.join(new_df)

### 5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN. Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [141]:
delays=pd.DataFrame(df['RecentDelays'].values.tolist(),columns=['Delay_1','Delay_2','Delay_3'])
df=df.join(delays)
df.drop(columns='RecentDelays',inplace=True)

### Airline column is not cleaned, hence I proactively cleaned it, even it though it was not asked

In [142]:
def airline_clean(column_name):
    try:
        airline=column_name.to_list()

        new_airline=[re.sub('[^a-zA-Z]+', "",item) for item in airline]
        new_airline=pd.DataFrame({"Air Carrier":new_airline})
    except Exception as e:
        print(e)
        
airline_clean(df['Airline'])
df=df.join(new_airline,how='inner')

In [143]:
df.drop(columns='Airline',inplace=True)

In [145]:
print('Final cleaned Dataset')
print("-"*25,sep=" ")
df

Final cleaned Dataset
-------------------------


,FlightNumber,From,To,Delay_1,Delay_2,Delay_3,Air Carrier
0,10045,London,Paris,23.0,47.0,NaN,KLM
1,10055,Madrid,Milan,NaN,NaN,NaN,AirFrance
2,10065,London,Stockholm,24.0,43.0,87.0,BritishAirways
3,10075,Budapest,Paris,13.0,NaN,NaN,AirFrance
4,10085,Brussels,London,67.0,32.0,NaN,SwissAir
